In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras import layers
from datasets import load_dataset

dataset = load_dataset("mstz/heart_failure")["train"]
df = pd.DataFrame(dataset)

df.head()

,age,has_anaemia,creatinine_phosphokinase_concentration_in_blood,has_diabetes,heart_ejection_fraction,has_high_blood_pressure,platelets_concentration_in_blood,serum_creatinine_concentration_in_blood,serum_sodium_concentration_in_blood,is_male,is_smoker,days_in_study,is_dead
0,75,False,582.0,False,20.0,True,265000.00,1.9,130.0,True,False,4,1
1,55,False,7861.0,False,38.0,False,263358.03,1.1,136.0,True,False,6,1
2,65,False,146.0,False,20.0,False,162000.00,1.3,129.0,True,True,7,1
3,50,True,111.0,False,20.0,False,210000.00,1.9,137.0,True,False,7,1
4,65,True,160.0,True,20.0,False,327000.00,2.7,116.0,False,False,8,1


In [12]:
# Normalize columns and convert bools to 0-1

def absolute_maximum_scale(series):
    return series / series.abs().max()

for col, col_type in zip(df.columns, df.dtypes):
    if col_type == bool:
        df[col] = df[col].map(float)
    else:
        df[col] = absolute_maximum_scale(df[col])

df.head()

,age,has_anaemia,creatinine_phosphokinase_concentration_in_blood,has_diabetes,heart_ejection_fraction,has_high_blood_pressure,platelets_concentration_in_blood,serum_creatinine_concentration_in_blood,serum_sodium_concentration_in_blood,is_male,is_smoker,days_in_study,is_dead
0,0.789474,0.0,0.074036,0.0,0.250,1.0,0.311765,0.202128,0.878378,1.0,0.0,0.014035,1.0
1,0.578947,0.0,1.000000,0.0,0.475,0.0,0.309833,0.117021,0.918919,1.0,0.0,0.021053,1.0
2,0.684211,0.0,0.018573,0.0,0.250,0.0,0.190588,0.138298,0.871622,1.0,1.0,0.024561,1.0
3,0.526316,1.0,0.014120,0.0,0.250,0.0,0.247059,0.202128,0.925676,1.0,0.0,0.024561,1.0
4,0.684211,1.0,0.020354,1.0,0.250,0.0,0.384706,0.287234,0.783784,0.0,0.0,0.028070,1.0


In [13]:
# Split input and output (X and Y)

predict = 'is_dead'

X = np.array(df.drop(predict, axis=1)) # Features
y = np.array(df[predict]) # Labels


In [23]:
def build_model():
    input_tensor = layers.Input(shape=(12,))

    x = layers.Dense(12, activation='relu')(input_tensor)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(
        inputs=input_tensor,
        outputs=x
    )
    return model


model = build_model()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 12)]              0         
                                                                 
 dense_11 (Dense)            (None, 12)                156       
                                                                 
 dense_12 (Dense)            (None, 1)                 13        
                                                                 
Total params: 169 (676.00 Byte)
Trainable params: 169 (676.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(X, y, validation_split=0.1, epochs=50, batch_size=10)

Epoch 1/50
27/27 [==============================] - 0s 4ms/step - loss: 0.7232 - accuracy: 0.4833 - val_loss: 0.7031 - val_accuracy: 0.3333
Epoch 2/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6673 - accuracy: 0.5428 - val_loss: 0.5804 - val_accuracy: 0.8333
Epoch 3/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6461 - accuracy: 0.6729 - val_loss: 0.5184 - val_accuracy: 0.9667
Epoch 4/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6377 - accuracy: 0.6543 - val_loss: 0.4760 - val_accuracy: 1.0000
Epoch 5/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6312 - accuracy: 0.6617 - val_loss: 0.4538 - val_accuracy: 1.0000
Epoch 6/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6253 - accuracy: 0.6654 - val_loss: 0.4367 - val_accuracy: 1.0000
Epoch 7/50
27/27 [==============================] - 0s 1ms/step - loss: 0.6206 - accuracy: 0.6617 - val_loss: 0.4258 - val_accuracy: 1.0000
Epoch 8/50
27/27 [==

In [25]:
loss, accuracy = model.evaluate(X, y)
print('Accuracy: %.2f' % (accuracy*100))

10/10 [==============================] - 0s 1ms/step - loss: 0.4141 - accuracy: 0.8562
Accuracy: 85.62
